In [1]:
from pprint import pprint
import copy

i = 0
group = {}

def reset():
    global i
    i = 0
    createGroup()

def addPerson(name):
    group['balances'][name] = {}

    # link people
    for p in group['people']:
        group['balances'][name][p] = 0
        group['balances'][p][name] = 0

    group['ledger'][name] = 0
    # add person to group
    group['people'][name] = {'name': name}

def createGroup():
    # clear shit
    group.clear()

    # setup group
    group['ledger'] = {}
    group['balances'] = {}
    group['people'] = {}
    group['transactions'] = {}
    addPerson("person1")
    addPerson("person2")
    addPerson("person3")

createGroup()
pprint(group)

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {}}


In [152]:
def add_transaction(who_got_how_much, who_paid_how_much):
    global i
    paid_total = sum(who_paid_how_much.values())
    used_total = sum(who_got_how_much.values())
    assert paid_total == used_total

    ledger_deltas = {}
    people_involved = set(who_got_how_much.keys()).union(set(who_paid_how_much.keys()))
    for p in people_involved:
        # update ledger
        used, paid = 0, 0
        if p in who_got_how_much:
            used = who_got_how_much[p]
        if p in who_paid_how_much:
            paid = who_paid_how_much[p]
        ledger_deltas[p] = paid - used

    # update ledger
    for k, v in ledger_deltas.items():
        group['ledger'][k] += v

    transaction = {'id': i, 'who_used': who_got_how_much.copy(), 'who_paid': who_paid_how_much.copy()}

    # init deltas
    balance_deltas = {}
    for p1 in people_involved:
        balance_deltas[p1] = {}
        for p2 in people_involved:
            if p1 != p2:
                balance_deltas[p1][p2] = 0

    # calc deltas and transactions
    for p_paid in who_paid_how_much.keys():
        paid = ledger_deltas[p_paid]
        # calculate how much if any this person has pair
        if p_paid in who_got_how_much:
            paid -= who_got_how_much[p_paid]
        for p_used in who_got_how_much.keys():
            # if the person that paid is not the same as the person that used
            if p_paid != p_used:
                if ledger_deltas[p_used] < 0 :
                    # if the amount paid is greater than the amount used
                    if ledger_deltas[p_used] + paid > 0:
                        group['balances'][p_paid][p_used] -= ledger_deltas[p_used]
                        group['balances'][p_used][p_paid] += ledger_deltas[p_used]
                        balance_deltas[p_paid][p_used] -= ledger_deltas[p_used]
                        balance_deltas[p_used][p_paid] += ledger_deltas[p_used]
                    # if the amount paid is equal to or less than the amount used
                    else:
                        paid = paid if paid > 0 else -paid
                        group['balances'][p_paid][p_used] += paid
                        group['balances'][p_used][p_paid] -= paid
                        balance_deltas[p_paid][p_used] += paid
                        balance_deltas[p_used][p_paid] -= paid

    # save deltas in transaction for later
    transaction['balance_deltas'] = balance_deltas
    transaction['ledger_deltas'] = ledger_deltas

    # save transaction
    group['transactions'][i] = transaction
    i += 1
    pprint(group)

    return transaction




In [129]:
reset()
t1 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person1': 20,
})


deltas
{'person1': {}}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}}}}


In [130]:
t2 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 20,
})

deltas
{'person1': {'person2': -20}, 'person2': {'person1': 20}}
{'balances': {'person1': {'person2': -20, 'person3': 0},
              'person2': {'person1': 20, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': -20, 'person2': 20, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}}}}


In [131]:
t2 = add_transaction(who_got_how_much={
    'person2': 20,
}, who_paid_how_much={
    'person1': 20,
})

deltas
{'person1': {'person2': 20}, 'person2': {'person1': -20}}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'deltas': {'person1': {'person2': 20},
                                 'person2': {'person1': -20}},
                

In [132]:
t3 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 10,
    'person3': 10,
})

deltas
{'person1': {'person2': -10, 'person3': -10},
 'person2': {'person1': 10, 'person3': 0},
 'person3': {'person1': 10, 'person2': 0}}
{'balances': {'person1': {'person2': -10, 'person3': -10},
              'person2': {'person1': 10, 'person3': 0},
              'person3': {'person1': 10, 'person2': 0}},
 'ledger': {'person1': -20, 'person2': 10, 'person3': 10},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'deltas': {'person1': {'person2': 

In [133]:
t4 = add_transaction(who_got_how_much={
    'person2': 10,
    'person3': 10,
}, who_paid_how_much={
    'person1': 20,
})

deltas
{'person1': {'person2': 10, 'person3': 10},
 'person2': {'person1': -10, 'person3': 0},
 'person3': {'person1': -10, 'person2': 0}}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'deltas': {'person1': {'person2': 20},
     

In [134]:
t5 = add_transaction(who_got_how_much={
    'person2': 30,
}, who_paid_how_much={
    'person2': 30,
})

deltas
{'person2': {}}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'deltas': {'person1': {'person2': 20},
                                 'person2': {'person1': -20}},
                      'id': 2,
                      'who_

In [135]:
t5 = add_transaction(who_got_how_much={
    'person2': 30,
    'person3': 30,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

deltas
{'person2': {'person3': 0}, 'person3': {'person2': 0}}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'deltas': {'person1': {'person2': 20},
                                 'person2': {'person1': -20}},
                   

In [136]:
t6 = add_transaction(who_got_how_much={
    'person2': 20,
    'person3': 40,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

deltas
{'person2': {'person3': 10}, 'person3': {'person2': -10}}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 10},
              'person3': {'person1': 0, 'person2': -10}},
 'ledger': {'person1': 0, 'person2': 10, 'person3': -10},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'deltas': {'person1': {}},
                      'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'deltas': {'person1': {'person2': -20},
                                 'person2': {'person1': 20}},
                      'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'deltas': {'person1': {'person2': 20},
                                 'person2': {'person1': -20}},
          

In [156]:
t6 = add_transaction(who_got_how_much={
    'person2': 40,
    'person3': 20,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': -20},
              'person3': {'person1': 0, 'person2': 20}},
 'ledger': {'person1': 0, 'person2': -20, 'person3': 20},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'balance_deltas': {'person2': {'person3': -10},
                                         'person3': {'person2': 10}},
                      'id': 0,
                      'ledger_deltas': {'person2': -10, 'person3': 10},
                      'who_paid': {'person2': 30, 'person3': 30},
                      'who_used': {'person2': 40, 'person3': 20}},
                  1: {'balance_deltas': {'person2': {'person3': -10},
                                         'person3': {'person2': 10}},
                      'id': 1,
                      'ledger_deltas': {'person2': -10, 'person3': 10},
                     

In [169]:
def update_transaction(t_id, who_got_how_much, who_paid_how_much):
    # first we need to undo the transaction by performing the opposite
    transaction = group['transactions'][t_id]
    balance_deltas = transaction['balance_deltas']
    ledger_deltas = transaction['ledger_deltas']
    people_involved = balance_deltas.keys()
    pprint(ledger_deltas)
    for p1 in people_involved:
        for p2 in people_involved:
            if p1 != p2:
                group['balances'][p1][p2] -= balance_deltas[p1][p2]
        group['ledger'][p1] -= ledger_deltas[p1]


    # delete the old transaction
    group['transactions'].pop(t_id)

    # create and return the new transaction
    return add_transaction(who_got_how_much=who_got_how_much, who_paid_how_much=who_paid_how_much)

In [170]:
reset()
t1 = add_transaction(who_got_how_much={
    'person2': 40,
    'person3': 20,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})


{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': -10},
              'person3': {'person1': 0, 'person2': 10}},
 'ledger': {'person1': 0, 'person2': -10, 'person3': 10},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'balance_deltas': {'person2': {'person3': -10},
                                         'person3': {'person2': 10}},
                      'id': 0,
                      'ledger_deltas': {'person2': -10, 'person3': 10},
                      'who_paid': {'person2': 30, 'person3': 30},
                      'who_used': {'person2': 40, 'person3': 20}}}}


In [171]:
t1 = update_transaction(t1['id'],
    who_got_how_much={
        'person2': 20,
        'person3': 40,
    }, who_paid_how_much={
        'person2': 30,
        'person3': 30,
    })

{'person2': -10, 'person3': 10}
{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 10},
              'person3': {'person1': 0, 'person2': -10}},
 'ledger': {'person1': 0, 'person2': 10, 'person3': -10},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {1: {'balance_deltas': {'person2': {'person3': 10},
                                         'person3': {'person2': -10}},
                      'id': 1,
                      'ledger_deltas': {'person2': 10, 'person3': -10},
                      'who_paid': {'person2': 30, 'person3': 30},
                      'who_used': {'person2': 20, 'person3': 40}}}}
